In [2]:
import json
import os
import datetime as dt 
from tqdm import tqdm
import pandas as pd 
import numpy as np
import time

In [3]:
# 文件名按照字符串取第一位置的字符串
def f2cat(filename :str) -> str:
    return filename.split('.')[0]

class Simplified():
    def __init__(self, input_path = '/share/data/quickdraw-doodle-recognition/train_simplified/'):
        self.input_path = input_path

    # 提取所有的label
    def list_all_categories(self):
        files = os.listdir(self.input_path)
        return sorted([f2cat(f) for f in files], key = str.lower)

    def read_training_csv(self, category, nrows = None, usecols = None, drawing_transform = False):
        df = pd.read_csv(
            os.path.join(self.input_path, category + '.csv'),
            nrows = nrows, parse_dates=['timestamp'], usecols = usecols
        )

        if drawing_transform:
            df['drawing'] = df['drawing'].apply(json.loads)
        return df

start = dt.datetime.now()

# 加载数据路径
s = Simplified()
NVSVS = 100

categories = s.list_all_categories()
print(len(categories))

340


In [4]:
for y, cat in tqdm(enumerate(categories)):
    df = s.read_training_csv(cat)
    df['y'] = y
    df['cv'] = (df.key_id// 10**7) % NVSVS # 处理成100个小文件
    # 写入数据
    for k in range(NVSVS):
        filename = '/data/python/tensorflow/shuffle_data/train_k{}.csv'.format(k)
        chunk = df[df.cv == k]
        chunk = chunk.drop(['key_id'], axis = 1)
        if y == 0: 
            chunk.to_csv(filename, index = False)
        else:
            chunk.to_csv(filename, mode = 'a', header = False,index = False)

340it [15:53,  2.80s/it]


In [5]:
# 对csv数据打乱，进行压缩并存储，删除原有数据
for k in tqdm(range(NVSVS)):
    filename = '/data/python/tensorflow/shuffle_data/train_k{}.csv'.format(k)
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        df['rnd'] = np.random.rand(len(df))
        df = df.sort_values(by = 'rnd').drop('rnd',axis = 1) # 对数据做随机打乱
        df.to_csv('/data/python/tensorflow/shuffle_data_gzip/train_k{}.csv.gz'.format(k), compression = 'gzip',index = False)
        print(time.strftime('%Y-%m-%d %H:%M:%S'),'\t',f'第{k}个数据压缩成功')
        #os.remove(filename)
print(df.shape)

end = dt.datetime.now()
print('lastest run {}.\nTotal time {}s'.format(end, (end - start).seconds))

100%|██████████| 100/100 [3:35:47<00:00, 129.47s/it]2020-09-17 12:53:41 	 第99个数据压缩成功
(497740, 7)
lastest run 2020-09-17 12:53:41.449062.
Total time 13901s

